In [18]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

## Extract and Transform

In [19]:
r = requests.get('http://swapi.dev/api/')

r.json()

{'people': 'https://swapi.dev/api/people/',
 'planets': 'https://swapi.dev/api/planets/',
 'films': 'https://swapi.dev/api/films/',
 'species': 'https://swapi.dev/api/species/',
 'vehicles': 'https://swapi.dev/api/vehicles/',
 'starships': 'https://swapi.dev/api/starships/'}

In [20]:
def get_full_type_data(base_url):
    data =  requests.get(base_url)
    all_data = []
    data_num = 0
    for i in range(200):
        if data_num < data.json()['count']+1:
            trial=requests.get(base_url+str(i))
            if str(trial) != '<Response [404]>':
                all_data.append(trial.json())
                data_num+=1
    all_data = pd.json_normalize(all_data)
    return all_data

In [21]:
def df_columns_to_numeric(df,columns):
    for column_i in columns:
        unusual_index_list = df.index[df[column_i].str.contains("\D")].tolist()
        # Rule out the unusual charactor in a numeric column
        if len(unusual_index_list)>0:
            df.loc[unusual_index_list,column_i]=np.nan
        df[column_i]=pd.to_numeric(df[column_i])
    return df

In [22]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected stage_dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [23]:
def replace_nans(x):
    if x in ['n/a', 'none','unknown','None']:
        return np.nan
    elif x is None:
        return np.nan
    else:
        return x

In [24]:
people = get_full_type_data('https://swapi.dev/api/people/')

In [25]:
#Check number and dimension of data
people.shape

(82, 16)

In [26]:
#show the dataset 
people.head(3)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/


In [27]:
def get_ids_from_urls(x):
    # print(x)
    if x is not np.nan:
        ids=[]
        if type(x) is list:
            if len(x)==0:
                return np.nan
            else:
                for url_i in x:
                    ids.append(re.findall(r'\d+',url_i)[0])
                return ids
        else:
            return re.findall(r'\d+',x)[0]
    else:
        return np.nan


In [28]:
# Featuring

people['id'] = people['url'].apply(lambda x: get_ids_from_urls(x))

people['planet_ids'] = people['homeworld'].apply(lambda x: get_ids_from_urls(x))

people['films_ids'] = people['films'].apply(lambda x: get_ids_from_urls(x))

people['species_ids'] = people['species'].apply(lambda x: get_ids_from_urls(x))

people['vehicles_ids'] = people['vehicles'].apply(lambda x: get_ids_from_urls(x))

people['starships_ids'] = people['starships'].apply(lambda x: get_ids_from_urls(x))



In [29]:
# Replace all kinds of nan annotation with proper nan
for column_i in ['height','mass','hair_color','skin_color','eye_color','birth_year','gender']:
    people[column_i]=people[column_i].apply(lambda x:replace_nans(x))

In [30]:
people['film_num']=people['films'].apply(lambda x:len(x))

people['vehicles_num']=people['vehicles'].apply(lambda x:len(x))

people['starships_num']=people['starships'].apply(lambda x:len(x))

people=people.replace('unknown',np.nan)

# There some error in the mass
people['mass']=people['mass'].apply(lambda x: str(x).replace(',',''))

people[['mass','height']]=people[['mass','height']].astype('float')

people['bmi']=people['mass']/(people['height']/100)**2

In [31]:
people['male'] = people['gender'].apply(lambda x: 1 if x=='male' else 0)

In [32]:
film_people = people.explode('films_ids')[['id','films_ids']]
film_people=film_people.rename({'id': 'people_ids'}, axis='columns')
film_people = film_people.dropna(axis=0)

species_people = people.explode('species_ids')[['id','species_ids']]
species_people=species_people.rename({'id': 'people_ids'}, axis='columns')
species_people = species_people.dropna(axis=0)

vehicles_people = people.explode('vehicles_ids')[['id','vehicles_ids']]
vehicles_people=vehicles_people.rename({'id': 'people_ids'}, axis='columns')
vehicles_people = vehicles_people.dropna(axis=0)

starships_people = people.explode('starships_ids')[['id','starships_ids']]
starships_people=starships_people.rename({'id': 'people_ids'}, axis='columns')
starships_people = starships_people.dropna(axis=0)


In [33]:
people = people.drop(columns=['vehicles','starships','films','url','homeworld','species','films_ids','species_ids','vehicles_ids','starships_ids'])
people.head(3)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,created,edited,id,planet_ids,film_num,vehicles_num,starships_num,bmi,male
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19BBY,male,2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,1,1,4,2,2,26.027582,1
1,C-3PO,167.0,75.0,NaN,gold,yellow,112BBY,NaN,2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,2,1,6,0,0,26.892323,0
2,R2-D2,96.0,32.0,NaN,"white, blue",red,33BBY,NaN,2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,3,8,6,0,0,34.722222,0


Planets

In [34]:
planets_df = get_full_type_data('http://swapi.dev/api/planets/')
print("Number of planets",planets_df.shape[0])

Number of planets 60


In [37]:
for column_i in list(planets_df.columns):
    planets_df[column_i]=planets_df[column_i].apply(lambda x:replace_nans(x))
planet_numeric_cols=['rotation_period','orbital_period','diameter','surface_water','population']
planets_df[planet_numeric_cols] = planets_df[planet_numeric_cols].astype('float')

planets_df['id'] = planets_df['url'].apply(lambda x: get_ids_from_urls(x))

planets_df['people_ids'] = planets_df['residents'].apply(lambda x: get_ids_from_urls(x))

planets_df['films_ids'] = planets_df['films'].apply(lambda x: get_ids_from_urls(x))

planets_df.head(3)


,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url,id,people_ids,films_ids
0,Tatooine,23.0,304.0,10465.0,arid,1 standard,desert,1.0,2.000000e+05,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/,1,"[1, 2, 4, 6, 7, 8, 9, 11, 43, 62]","[1, 3, 4, 5, 6]"
1,Alderaan,24.0,364.0,12500.0,temperate,1 standard,"grasslands, mountains",40.0,2.000000e+09,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/,2,"[5, 68, 81]","[1, 6]"
2,Yavin IV,24.0,4818.0,10200.0,"temperate, tropical",1 standard,"jungle, rainforests",8.0,1.000000e+03,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/,3,NaN,[1]


Film

In [45]:
films = get_full_type_data('http://swapi.dev/api/films/')
print("Number of films",films.shape[0])

Number of films 6


In [46]:
for column_i in list(films.columns):
    films[column_i]=films[column_i].apply(lambda x:replace_nans(x))

films['id'] = films['url'].apply(lambda x: get_ids_from_urls(x))

films['people_ids'] = films['characters'].apply(lambda x: get_ids_from_urls(x))

films['planets_ids'] = films['planets'].apply(lambda x: get_ids_from_urls(x))

films['starships_ids'] = films['starships'].apply(lambda x: get_ids_from_urls(x))

films['vehicles_ids'] = films['vehicles'].apply(lambda x: get_ids_from_urls(x))

films['species_ids'] = films['species'].apply(lambda x: get_ids_from_urls(x))


In [47]:
films['count_characters']=films['characters'].apply(lambda x: len(x))
films['count_planets']=films['planets'].apply(lambda x: len(x))
films['count_starships']=films['starships'].apply(lambda x: len(x))
films['count_vehicles']=films['vehicles'].apply(lambda x: len(x))
films['count_species']=films['species'].apply(lambda x: len(x))

In [48]:
planet_film = films.explode('planets_ids')[['id','planets_ids']]
planet_film=planet_film.rename({'id': 'film_ids'}, axis='columns')
planet_film = planet_film.dropna(axis=0)

starships_film = films.explode('starships_ids')[['id','starships_ids']]
starships_film=starships_film.rename({'id': 'film_ids'}, axis='columns')
starships_film = starships_film.dropna(axis=0)

vehicles_film = films.explode('vehicles_ids')[['id','vehicles_ids']]
vehicles_film=vehicles_film.rename({'id': 'film_ids'}, axis='columns')
vehicles_film = vehicles_film.dropna(axis=0)

species_film = films.explode('species_ids')[['id','species_ids']]
species_film=species_film.rename({'id': 'film_ids'}, axis='columns')
species_film = species_film.dropna(axis=0)

In [49]:
films=films.drop(columns=['characters','planets','starships','vehicles','species','people_ids','planets_ids','starships_ids','vehicles_ids','species_ids'])
films.head(3)

,title,episode_id,opening_crawl,director,producer,release_date,created,edited,url,id,count_characters,count_planets,count_starships,count_vehicles,count_species
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/,1,18,3,8,4,5
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/,2,16,4,9,6,5
2,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,2014-12-18T10:39:33.255000Z,2014-12-20T09:48:37.462000Z,https://swapi.dev/api/films/3/,3,20,5,12,8,9


Speicies

In [50]:
species = get_full_type_data('http://swapi.dev/api/species/')
print("Number of films",species.shape[0])

Number of films 37


In [51]:
for column_i in list(species.columns):
    species[column_i]=species[column_i].apply(lambda x:replace_nans(x))

species['id'] = species['url'].apply(lambda x: get_ids_from_urls(x))

species['people_ids'] = species['people'].apply(lambda x: get_ids_from_urls(x))

species['planets_ids'] = species['homeworld'].apply(lambda x: get_ids_from_urls(x))

species['people_ids'] = species['people'].apply(lambda x: get_ids_from_urls(x))



In [52]:
planet_species = species.explode('planets_ids')[['id','planets_ids']]
planet_species=planet_species.rename({'id': 'species_ids'}, axis='columns')
planet_species = planet_species.dropna(axis=0)

In [53]:
species=species.drop(columns=['people','url','homeworld','people','people_ids','planets_ids','films'])
species.head(3)

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,language,created,edited,id
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,Galactic Basic,2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,1
1,Droid,artificial,sentient,NaN,NaN,NaN,NaN,indefinite,NaN,2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,2
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,Shyriiwook,2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,3


Vehicles

In [54]:
# Scraping all vehi
vehicles_df = get_full_type_data('http://swapi.dev/api/vehicles/')
print("Number of vehicles",vehicles_df.shape[0])

Number of vehicles 39


In [55]:
for column_i in list(vehicles_df.columns):
    vehicles_df[column_i]=vehicles_df[column_i].apply(lambda x:replace_nans(x))

vehicles_df['id'] = vehicles_df['url'].apply(lambda x: get_ids_from_urls(x))

vehicles_df['people_ids'] = vehicles_df['pilots'].apply(lambda x: get_ids_from_urls(x))

vehicles_df['films_ids'] = vehicles_df['films'].apply(lambda x: get_ids_from_urls(x))



In [56]:
vehicles_df=vehicles_df.drop(columns=['pilots','url','films','people_ids','films_ids'])
vehicles_df.head(3)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,created,edited,id
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,4
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,2014-12-10T16:01:52.434000Z,2014-12-20T21:30:21.665000Z,6
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NaN,repulsorcraft,2014-12-10T16:13:52.586000Z,2014-12-20T21:30:21.668000Z,7


Starships

In [57]:
# Scraping all attributes of starships 

starships_df = get_full_type_data('http://swapi.dev/api/starships/')

print("Number of starships",starships_df.shape[0])

Number of starships 36


In [58]:
for column_i in list(starships_df.columns):
    starships_df[column_i]=starships_df[column_i].apply(lambda x:replace_nans(x))
num_cols=['cost_in_credits','length','max_atmosphering_speed','crew','cargo_capacity','passengers','hyperdrive_rating','MGLT']
for col in num_cols:
    # For all columns in numeric vars list we featured them to right structure
     #-I find that separator is "," inplace of "."
     #- '-' between numbers that is supposed to unique number
     #- 'km' in number, need to fix that to allow conversion
    
    starships_df[col]=starships_df[col].apply(lambda x: str(x).replace(',','.'))
    starships_df[col]=starships_df[col].apply(lambda x: str(x).replace('-',''))
    starships_df[col]=starships_df[col].apply(lambda x: str(x).replace('km',' '))
# Converting all numerics columns in right data type for displaying some statistics

starships_df[num_cols]=starships_df[num_cols].astype('float')


In [59]:
starships_df.head(3)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films,created,edited,url
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000.0,150.0,950.0,30165.00,600.0,3000000.0,1 year,2.0,60.0,corvette,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,https://swapi.dev/api/starships/2/
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000.0,1.6,975.0,47.06,NaN,36000000.0,2 years,2.0,60.0,Star Destroyer,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,https://swapi.dev/api/starships/3/
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000.0,38.0,1000.0,5.00,75.0,180000.0,1 month,1.0,70.0,landing craft,[],[https://swapi.dev/api/films/1/],2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,https://swapi.dev/api/starships/5/


In [60]:
starships_df['id'] = starships_df['url'].apply(lambda x: get_ids_from_urls(x))

starships_df['people_ids'] = starships_df['pilots'].apply(lambda x: get_ids_from_urls(x))

starships_df['films_ids'] = starships_df['films'].apply(lambda x: get_ids_from_urls(x))

In [61]:
starships_df=starships_df.drop(columns=['pilots','url','films','people_ids','films_ids'])
starships_df.head(3)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,created,edited,id
0,CR90 corvette,CR90 corvette,Corellian Engineering Corporation,3500000.0,150.0,950.0,30165.00,600.0,3000000.0,1 year,2.0,60.0,corvette,2014-12-10T14:20:33.369000Z,2014-12-20T21:23:49.867000Z,2
1,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000.0,1.6,975.0,47.06,NaN,36000000.0,2 years,2.0,60.0,Star Destroyer,2014-12-10T15:08:19.848000Z,2014-12-20T21:23:49.870000Z,3
2,Sentinel-class landing craft,Sentinel-class landing craft,"Sienar Fleet Systems, Cyngus Spaceworks",240000.0,38.0,1000.0,5.00,75.0,180000.0,1 month,1.0,70.0,landing craft,2014-12-10T15:48:00.586000Z,2014-12-20T21:23:49.873000Z,5


## Load

In [62]:
import sqlite3

In [63]:
conn = sqlite3.connect('starwars_database1.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS people ({', '.join(people.columns)})")
conn.commit()
people.to_sql('people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films ({', '.join(films.columns)})")
conn.commit()
films.to_sql('films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS planets ({', '.join(planets_df.columns)})")
conn.commit()
films.to_sql('planets', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS species ({', '.join(species.columns)})")
conn.commit()
species.to_sql('species', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles ({', '.join(vehicles_df.columns)})")
conn.commit()
vehicles_df.to_sql('vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS starships ({', '.join(starships_df.columns)})")
conn.commit()
starships_df.to_sql('starships', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS film_people ({', '.join(film_people.columns)})")
conn.commit()
film_people.to_sql('film_people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS species_people ({', '.join(species_people.columns)})")
conn.commit()
species_people.to_sql('species_people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_people ({', '.join(vehicles_people.columns)})")
conn.commit()
vehicles_people.to_sql('vehicles_people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS planet_film ({', '.join(planet_film.columns)})")
conn.commit()
planet_film.to_sql('planet_film', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles_film ({', '.join(vehicles_film.columns)})")
conn.commit()
vehicles_film.to_sql('vehicles_film', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS species_film ({', '.join(species_film.columns)})")
conn.commit()
species_film.to_sql('species_film', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS planet_species ({', '.join(planet_species.columns)})")
conn.commit()
planet_species.to_sql('planet_species', conn, if_exists='replace', index = False)



36

## Analysis

In [64]:
con = sqlite3.connect('starwars_database1.db')

INNER JOIN


Find out the film with the highest average height

In [65]:
pd.read_sql(
    '''
        SELECT 
            f.title,
            avg(p.height) AS average_height
        FROM 
            film_people as fp
        INNER JOIN 
            people p ON fp.people_ids = p.id
        INNER JOIN
            films f ON f.id = fp.films_ids
        GROUP BY
            f.id
        ORDER BY 
            average_height DESC 
        LIMIT 5
    ''', 
    con
)

,title,average_height
0,Revenge of the Sith,177.764706
1,Attack of the Clones,174.950000
2,A New Hope,170.333333
3,The Phantom Menace,169.735294
4,The Empire Strikes Back,169.250000


RIGHT JOIN

get the total cost_in_credits in veihcles for all people
(Some people may not own vehicle, so use right join)

In [66]:
pd.read_sql(
    '''
        SELECT 
            p.name,
            sum(v.cost_in_credits) AS total_cost_in_credits
        FROM 
            vehicles_people as vp
        RIGHT JOIN 
            people p ON vp.people_ids = p.id
        INNER JOIN
            vehicles v ON v.id = vp.vehicles_ids
        GROUP BY
            p.id
        ORDER BY 
            total_cost_in_credits DESC
        LIMIT 5
    ''', 
    con
)

,name,total_cost_in_credits
0,Grievous,15000
1,Dooku,8000
2,Leia Organa,8000
3,Luke Skywalker,8000
4,Anakin Skywalker,5750


GROUP BY

Find out the film with the highest male ratio

In [67]:
pd.read_sql(
    '''
        SELECT 
            f.title,
            ROUND((sum(p.male)+0.0)/(sum(p.id)*100+0.0) ,5)*100 AS gender_ratio,
            sum(p.male) AS male_number,
            sum(p.id) AS total_number
        FROM 
            film_people as fp
        INNER JOIN 
            people p ON fp.people_ids = p.id
        INNER JOIN
            films f ON f.id = fp.films_ids
        GROUP BY
            f.id
        ORDER BY 
            gender_ratio DESC
        LIMIT 5
    ''', 
    con
)

,title,gender_ratio,male_number,total_number
0,The Empire Strikes Back,0.052,12,231
1,A New Hope,0.049,12,243
2,Return of the Jedi,0.041,15,364
3,The Phantom Menace,0.021,27,1312
4,Revenge of the Sith,0.016,23,1428


Note, a large group of people are missing gender info. The ratio might be wrong

HAVING

FIND OUT the film with fit people (average BMI in normal range)

In [68]:
pd.read_sql(
    '''
        SELECT 
            f.title,
            avg(p.bmi) AS average_height
        FROM 
            film_people as fp
        INNER JOIN 
            people p ON fp.people_ids = p.id
        INNER JOIN
            films f ON f.id = fp.films_ids
        GROUP BY
            f.id
        HAVING 
            average_height > 18.5 AND average_height < 24.9
        LIMIT 5
    ''', 
    con
)

,title,average_height
0,Attack of the Clones,22.910141
1,Revenge of the Sith,24.127241


CTE 

Present the respective number of films where male, female and hermaphrodite charactors are in.

In [77]:
pd.read_sql(
    '''
        WITH male_films AS (
            SELECT 
                'male' as gender,
                count(p.id) AS films_num
            FROM 
                film_people as fp
            INNER JOIN 
                people p ON fp.people_ids = p.id
            INNER JOIN
                films f ON f.id = fp.films_ids
            WHERE
                p.gender = 'male'
        ),
                    
        female_films AS (
            SELECT 
                'female' as gender,
                count(p.id) AS films_num
            FROM 
                film_people as fp
            INNER JOIN 
                people p ON fp.people_ids = p.id
            INNER JOIN
                films f ON f.id = fp.films_ids
            WHERE
                p.gender = 'female'    
        ),
        hermaphrodite_films AS (
            SELECT 
                'hermaphrodite' as gender,
                count(p.id) AS films_num
            FROM 
                film_people as fp
            INNER JOIN 
                people p ON fp.people_ids = p.id
            INNER JOIN
                films f ON f.id = fp.films_ids
            WHERE
                p.gender = 'hermaphrodite'    
        )
        
        SELECT 
            *
        FROM
            male_films 
        
        UNION
        
        SELECT 
            *
        FROM 
            female_films
        
        UNION

        SELECT 
            *
        FROM 
            hermaphrodite_films
    ''', 
    con
)

,gender,films_num
0,female,32
1,hermaphrodite,3
2,male,113
